In [1]:
from __future__ import division
%pylab inline
import pandas as pd
import os
import json
from collections import defaultdict, Counter
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


# 1. Number of sites per CDN and AS

In [4]:
df_asn_cdn = pd.read_pickle('output/df_asn_cdn.pkl')
df_site_cdn_method_map = pd.read_pickle('output/df_site_cdn_method_map.pkl')

In [6]:
df_asn_cdn.head()

,rank,site,IP,ASN,cdn_parsed,cdn_whois,cdn
0,1,google.com,172.217.160.238,15169,Google,Google Cloud,Google
1,2,youtube.com,172.217.31.14,15169,Google,Google Cloud,Google
2,3,facebook.com,157.240.25.35,32934,Facebook,Facebook,Facebook
3,4,baidu.com,123.125.115.110,4808,False,False,False
4,5,wikipedia.org,103.102.166.224,14907,False,False,False


# 2. Site timings per CDN